In [1]:
import pandas as pd 
import numpy as np
from pandas_datareader import data as pdr
import yfinance as yf #yahoo finance stock data
import datetime
import matplotlib.pyplot as plt
from math import sqrt

from sklearn.linear_model import LinearRegression,ElasticNet
from sklearn.ensemble import RandomForestRegressor

stock="GOOG"
period=14

In [2]:
min=yf.download(tickers=stock,period="3d", interval="1m")
thirt_min=yf.download(tickers=stock,period="31d", interval="30m")
hour=yf.download(tickers=stock,period="122d", interval="1h")
daily=yf.download(tickers=stock,period="765d", interval="1d")
week=yf.download(tickers=stock,period="3650d", interval="1wk")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [3]:
def Prepare_dataset(df):
    from sklearn.model_selection import train_test_split

    typ_price=(df.Close + df.High+df.Low)/3
    mf=typ_price*df.Volume

    pos_flow=[]
    neg_flow=[]

    for i in range(0,len(typ_price)):
        if typ_price[i]>typ_price[i-1]:
            pos_flow.append(mf[i-1])
            neg_flow.append(0)
        elif typ_price[i]<typ_price[i-1]:
            pos_flow.append(0)
            neg_flow.append(mf[i-1])
        else:
            pos_flow.append(0)
            neg_flow.append(0)
        
    pos_mf=[]
    neg_mf=[]

    for i in range(period-1,len(pos_flow)):
        pos_mf.append(sum(pos_flow[i+1-period:i]))

    for i in range(period-1,len(neg_flow)):
        neg_mf.append(sum(neg_flow[i+1-period:i]))

    mf_ratio=np.array(pos_mf)/np.array(neg_mf)

    mfi=100-100/(1+mf_ratio)
    df=df.iloc[13:,:]

    df["MFI"]=mfi

    y=df.iloc[:-1,:]["Adj Close"].shift(-1)
    y.fillna(method='ffill', inplace=True)
    X=df.iloc[:-1,:].fillna(method='ffill')

    return train_test_split(X, y, test_size=0.4, random_state=0)

def Rmse(y,y_pred):
    return np.sqrt(((y_pred - y) ** 2).mean())

In [4]:
datasets={}
datasets["1m"]=Prepare_dataset(min)
datasets["30m"]=Prepare_dataset(thirt_min)
datasets["1h"]=Prepare_dataset(hour)
datasets["D"]=Prepare_dataset(daily)
datasets["W"]=Prepare_dataset(week)

In [5]:
from sklearn.linear_model import LinearRegression

In [6]:
result=pd.DataFrame(columns=["1m","30m","1h","D","W"])

for key in datasets:
    X_train=datasets[key][0]
    X_test=datasets[key][1]
    y_train=datasets[key][2]
    y_test=datasets[key][3]

    elastic=ElasticNet().fit(X_train, y_train)
    elastic_preds=elastic.predict(X_test)

    result.loc["Elastic net",key]=[Rmse(y_test,elastic_preds)]

    rf=RandomForestRegressor(n_estimators=100,max_depth=12,random_state=0).fit(X_train, y_train)
    rf_preds=rf.predict(X_test)

    result.loc["Random forest",key]=[Rmse(y_test,rf_preds)]


In [7]:
result

,1m,30m,1h,D,W
Elastic net,[1.1562700404233666],[7.940113179513342],[8.284515424541974],[24.9314827888373],[26.668517552184422]
Random forest,[1.3052370753951565],[8.495361937752566],[10.921542144373083],[26.444181176480715],[28.439515571050205]
